### Welcome to the ProtoSyn.jl examples

# 3 - Molecular Manipulation

One of the main applications of ProtoSyn is the easy manipulation of molecular structures: add and remove residues and whole loops, mutate aminoacids, bond and unbond atoms or completly remove them from the structure. These are just some examples of the manipulations possible with the ProtoSyn framework. In the next examples we will explore a few of these tasks. Despite most of the methods used in this example having a Core counter-part, for this example, we will focus on the Peptides module, as it provides specialized methods for dealing with peptidic chains. We will start by creating a new Pose from scratch, employing the `build` function from the `Builder` submodule.

In [ ]:
using ProtoSyn

ProtoSyn allows users to create new molecular structures from scratch, using L-Grammars as residue libraries. An L-Grammar serves as a bank of templates from which the `build` method can retrieve each residue to create a new Pose. Note that this new Pose will have a linear conformation. For more details, check the documentation.

In [ ]:
res_lib = ProtoSyn.Peptides.grammar;

In [ ]:
pose = ProtoSyn.Peptides.build(res_lib, seq"AAAAAA")

We can export and visualize the resulting structure in our preferred visualization software. In this example, we will use the Julia's package Bio3DView. Note that this visualization does not include the hydrogens.

In [ ]:
using Bio3DView

In [ ]:
ProtoSyn.write(pose, "output/example3.pdb")

In [ ]:
style = Style("stick")
viewfile("output/example3.pdb", style = style)

## a) Appending and inserting residues to the molecular structure

From the same L-Grammar used to build the initial peptide, we can also append 1 or more new residues to the end of an existing structure (built or loaded from a file).

In [ ]:
ProtoSyn.append_fragment!(pose, pose.graph[1][end], res_lib, seq"FFF")

In [ ]:
ProtoSyn.write(pose, "output/example3.pdb")
viewfile("output/example3.pdb", style = style)

In a similar fashion, we can also insert 1 or more new residues in the middle of the structure, displacing all pre-existing residues downstream.

In [ ]:
ProtoSyn.Peptides.insert_fragment!(pose, pose.graph[1][4], res_lib, seq"FFF")

In [ ]:
ProtoSyn.write(pose, "output/example3.pdb")
viewfile("output/example3.pdb", style = style)

## b) Mutating an aminoacid

Another common task is to change the nature of an aminoacid, mutating it to another type. This is easilly achieved in ProtoSyn, using the `Peptides.mutate!` method.

In [ ]:
ProtoSyn.Peptides.mutate!(pose, pose.graph[1][4], res_lib, seq"K")

In [ ]:
ProtoSyn.write(pose, "output/example3.pdb")
viewfile("output/example3.pdb", style = style)

## c) Removing and adding sidechains

In certain simulations, especially when simulating large structures, it is common to employ coarse-grained methods to evaluate the energy of the system. In such cases, it can be helpfull to remove the sidechains in order to lower the degrees of freedom of the system. This can be done, in ProtoSyn, using the `Peptides.remove_sidechains!` method. At a latter point in the simulation, the sidechains can be recovered by using the `Peptides.add_sidechains!` method (although losing the initial conformation).

In [ ]:
ProtoSyn.Peptides.remove_sidechains!(pose, res_lib)

In [ ]:
ProtoSyn.write(pose, "output/example3.pdb")
viewfile("output/example3.pdb", style = style)

In [ ]:
ProtoSyn.Peptides.add_sidechains!(pose, res_lib)

In [ ]:
ProtoSyn.write(pose, "output/example3.pdb")
viewfile("output/example3.pdb", style = style)

Consider reading the documentation for more details!

## d) Applying a new Rotamer to the sidechains

A Rotamer is a set of dihedral angle values for each of the chi angles in the sidechain of an aminoacid. These can be organized in a rotamer library. Multiple instances of such libraries have been proposed in the past. ProtoSyn employs, by default, the Dunbrack Rotamer Library 2011, a backbone-dependent library. This means that available rotamers for a given aminoacid's sidechain are restricted by the current backbone phi and psi dihedral angles, based on the observed probabilities in nature. In ProtoSyn, a user is able to load this rotamer library and explore it freely to apply a new rotamer to a given sidechain, as exemplified bellow.

In [ ]:
rot_lib = ProtoSyn.Peptides.load_dunbrack()

Since this is a backbone-dependent library, we can find the current backbone phi and psi dihedral angle values using the `ProtoSyn.getdihedral` method. On top of that, using the dihedral selections, the `phi` and `psi` methods automatically retrieve the correct atom whose `:ϕ` field controls the phi and psi dihedral angle values, respectivelly.

In [ ]:
phi = ProtoSyn.getdihedral(pose.state, ProtoSyn.Peptides.phi(pose.graph[1][4]))
psi = ProtoSyn.getdihedral(pose.state, ProtoSyn.Peptides.psi(pose.graph[1][4]))
println("Phi: $phi | Psi: $psi")

In [ ]:
stack = rot_lib["PHE"][phi, psi]

The table above shows all available rotamers for the "PHE" dihedral (for example, for residue 5), for the specific phi and psi dihedral angle values, organized by natural probability of occurrence. We can now either randomly sample from this list or pick one of them in order to apply to the Pose. In this example, we will apply the most likely rotamer. Don't forget to sync! the new internal coordinate changes to cartesian coordinates (`ProtoSyn.write!` automatically syncs the Pose).

In [ ]:
ProtoSyn.Peptides.apply!(pose.state, stack[1], pose.graph[1][5])

In [ ]:
ProtoSyn.write(pose, "output/example3.pdb")
viewfile("output/example3.pdb", style = style)

## e) Remove an aminoacid

Using the `pop_residue!` method, users can remove specific Residues from a Pose.

In [ ]:
ProtoSyn.pop_residue!(pose, pose.graph[1][end])

In [ ]:
ProtoSyn.write(pose, "output/example3.pdb")
viewfile("output/example3.pdb", style = style)

It's also possible to remove a residue in the middle of a chain, braking it. In this case, the downstream residues are maintained in the current position but are attached to the Pose's root.

In [ ]:
ProtoSyn.pop_residue!(pose, pose.graph[1][6])

In [ ]:
ProtoSyn.write(pose, "output/example3.pdb")
viewfile("output/example3.pdb", style = style)

Since the downstream residue is now connected to the root, the directed graph is severed. As such, any upstream change, such as a dihedral rotation (as exemplified bellow), will only apply to the contiguous section of the Segment. Note that, in the snippet bellow, the `ProtoSyn.Units` module is used to simplify the writting of `rad2deg(90)` to simply `90°`.

In [ ]:
using ProtoSyn.Units

ProtoSyn.rotate_dihedral!(pose.state, ProtoSyn.Peptides.phi(pose.graph[1][2]), 90°)

In [ ]:
ProtoSyn.write(pose, "output/example3.pdb")
viewfile("output/example3.pdb", style = style)

## Conclusion

In this example, we took a look at some of the functions made available by ProtoSyn to help manipulate molecular structures, such as:
+ Appending and inserting new aminoacids
+ Mutating aminoacids
+ Removing and adding sidechains
+ Applying rotamers from a library
+ Removing residues
+ Applying specific dihedral rotations

Such set of tools is useful when setting up new starting points for simulations and prototyping new proteic designs, for example.